<span style="color:darkcyan;">**Téléchargement des bibliothèques**</span>

In [3]:
import pandas as pd

<span style="color:darkcyan;">**Importation de la base de données et création du data frame**</span>

In [9]:
df = pd.read_excel('/home/onyxia/work/Projet_Python-pour-la-data-science/données_vols.xlsx')
# Afficher les 5 premières lignes pour vérifier
print(df.head(5))

  Carrier Code    Date (MM/DD/YYYY)  Flight Number Tail Number  \
0           AA  2020-01-01 00:00:00              1      N110AN   
1           AA  2020-01-01 00:00:00              3      N111ZM   
2           AA  2020-01-01 00:00:00            111      N663AW   
3           AA  2020-01-01 00:00:00            117      N113AN   
4           AA  2020-01-01 00:00:00            179      N103NN   

  Destination Airport Scheduled departure time Actual departure time  \
0                 LAX                 07:30:00              07:30:00   
1                 LAX                 12:30:00              12:24:00   
2                 CLT                 12:00:00              13:11:00   
3                 LAX                 19:30:00              19:26:00   
4                 SFO                 10:30:00              10:25:00   

   Scheduled elapsed time (Minutes)  Actual elapsed time (Minutes)  \
0                               393                            404   
1                             

In [10]:
# Vérifier si le DataFrame contient des valeurs manquantes
print(df.isnull().values.any())

True


In [11]:
# Nombre de valeurs manquantes par colonne
print(df.isnull().sum())

Carrier Code                                  0
Date (MM/DD/YYYY)                             0
Flight Number                                 0
Tail Number                                 226
Destination Airport                           0
Scheduled departure time                      0
Actual departure time                         0
Scheduled elapsed time (Minutes)              0
Actual elapsed time (Minutes)                 0
Departure delay (Minutes)                     0
Wheels-off time                               0
Taxi-Out time (Minutes)                       0
Delay Carrier (Minutes)                       0
Delay Weather (Minutes)                       0
Delay National Aviation System (Minutes)      0
Delay Security (Minutes)                      0
Delay Late Aircraft Arrival (Minutes)         0
dtype: int64


<span style="color:lightpink;">**Commentaire de la sortie**</span>

<small>La seule variable qui présente des valeurs manquantes est **Tail Number**. Cependant il ne s'agit pas d'une variable explicative dans notre modèle, ainsi ces valeurs manquantes ne nécessite aucun traitement.</small>

In [8]:
# Filtrer les lignes avec des valeurs manquantes
missing_values_rows = df[df.isnull().any(axis=1)]

# Afficher les lignes avec des valeurs manquantes
print(missing_values_rows)

                                      Carrier Code    Date (MM/DD/YYYY)  \
13937                                           AA           03/18/2020   
14121                                           AA           03/19/2020   
14304                                           AA           03/20/2020   
14492                                           AA           03/21/2020   
15035                                           AA           03/24/2020   
...                                            ...                  ...   
35694                                           AA           07/29/2020   
35706                                           AA           07/29/2020   
49330                                           AA  2021-01-11 00:00:00   
57410                                          NaN                  NaN   
57411  SOURCE: Bureau of Transportation Statistics                  NaN   

       Flight Number Tail Number Destination Airport Scheduled departure time  \
13937          796

<span style="color:darkcyan;">**Extraction de la date du jour du vol de la colonne Date (MM/DD/YYYY)**</span>

<small>D'après la sortie du code précédent, nous pouvons clairement voir que la colonne **Date (MM/DD/YYYY)** ne contient pas uniquement la date du jour du vol mais aussi la chaîne de caractère **00:00:00** qui renvoie à **l'heure du début d'un jour**.</small>

In [12]:
# Extraire uniquement la date de la colonne Date (MM/DD/YYYY) dans la nouvelle colonne Date
df['Date'] = pd.to_datetime(df['Date (MM/DD/YYYY)']).dt.date  

# Afficher la colonne Date
print(df.Date)

0        2020-01-01
1        2020-01-01
2        2020-01-01
3        2020-01-01
4        2020-01-01
            ...    
57405    2023-12-31
57406    2023-12-31
57407    2023-12-31
57408    2023-12-31
57409    2023-12-31
Name: Date, Length: 57410, dtype: object


<span style="color:darkcyan;">**Création de la colonne Weekday_Flight**</span>

<small>Nous aimerions aussi voir si **le retard d'un vol peut être lié au jour de la semaine où le vol aura lieu**. Pour cela, il est nécessaire de créer une colonne contenant le jour de la semaine **(Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday)** correspondant à chaque vol de notre data frame.
</small>

In [13]:
# Convertir la colonne 'Date' en format datetime 
df['Date'] = pd.to_datetime(df['Date'])

# Créer la colonne Weekday_Flight avec les jours de la semaine correspondant aux dates des vols
df['Weekday_Flight'] = df['Date'].dt.day_name()

# Afficher Date et Weekday_Flight
print(df[['Date','Weekday_Flight']])

            Date Weekday_Flight
0     2020-01-01      Wednesday
1     2020-01-01      Wednesday
2     2020-01-01      Wednesday
3     2020-01-01      Wednesday
4     2020-01-01      Wednesday
...          ...            ...
57405 2023-12-31         Sunday
57406 2023-12-31         Sunday
57407 2023-12-31         Sunday
57408 2023-12-31         Sunday
57409 2023-12-31         Sunday

[57410 rows x 2 columns]


<span style="color:darkcyan;">**Création de la colonne Season**</span>

<small> La variable **"saison"** peut influencer les retards des vols en raison des **variations météorologiques** et des **volumes de trafic** spécifiques à chaque période de l'année. Par exemple, l’hiver apporte souvent des conditions difficiles comme la neige ou le brouillard, tandis que l’été, marqué par un trafic élevé, peut être perturbé par des orages ou des surcharges aéroportuaires. Ainsi, en tenant compte des saisons, il devient possible de mieux comprendre et anticiper les facteurs contribuant aux retards. </small>

In [14]:
# Définir une fonction qui détermine pour une date données la saison correspondante
def get_season(date):
    year = date.year
    if date >= pd.Timestamp(year=year, month=3, day=21) and date < pd.Timestamp(year=year, month=6, day=21):
        return 'Printemps'
    elif date >= pd.Timestamp(year=year, month=6, day=21) and date < pd.Timestamp(year=year, month=9, day=23):
        return 'Été'
    elif date >= pd.Timestamp(year=year, month=9, day=23) and date < pd.Timestamp(year=year, month=12, day=21):
        return 'Automne'
    else:
        return 'Hiver'


# Convertir la colonne Date en datetime 
df['Date'] = pd.to_datetime(df['Date'])

# Appliquer la fonction sur la colonne Date du dataframe
df['Season'] = df['Date'].apply(get_season)

# Afficher  des observations aléatoires du DataFrame
print(df.Season[1000:1010])
print(df.Season[2000:2010])


1000    Printemps
1001    Printemps
1002    Printemps
1003    Printemps
1004    Printemps
1005    Printemps
1006    Printemps
1007    Printemps
1008    Printemps
1009    Printemps
Name: Season, dtype: object
2000    Automne
2001    Automne
2002    Automne
2003    Automne
2004    Automne
2005    Automne
2006    Automne
2007    Automne
2008    Automne
2009    Automne
Name: Season, dtype: object


<span style="color:darkcyan;">**Création de notre variable cible Y=Delay**</span>

<small> Le **retard d'un vol au départ** se définit comme **l'écart entre l'heure de départ ou d'arrivée prévue d'un vol, telle qu'indiquée dans le programme, et l'heure réelle à laquelle le vol décolle**.. Un vol est souvent considéré en retard si cet écart dépasse un seuil défini, par exemple 5 minutes.
</small>

In [15]:
# Convertir les colonnes "Scheduled departure time" et "Actual departure time" en objets datetime
df['Scheduled departure time'] = pd.to_datetime(df['Scheduled departure time'], format='%H:%M:%S')
df['Actual departure time'] = pd.to_datetime(df['Actual departure time'], format='%H:%M:%S')

# Création de la variable cible "Retard en calculant la différence en minutes et appliquer la condition
df['Retard'] = ((df['Actual departure time'] - df['Scheduled departure time']).dt.total_seconds() / 60 > 5).astype(int)

# Afficher les colonnes "Actual departure time", "Scheduled departure time" et "Retard"
print(df[['Actual departure time', 'Scheduled departure time', 'Retard']])


      Actual departure time Scheduled departure time  Retard
0       1900-01-01 07:30:00      1900-01-01 07:30:00       0
1       1900-01-01 12:24:00      1900-01-01 12:30:00       0
2       1900-01-01 13:11:00      1900-01-01 12:00:00       1
3       1900-01-01 19:26:00      1900-01-01 19:30:00       0
4       1900-01-01 10:25:00      1900-01-01 10:30:00       0
...                     ...                      ...     ...
57405   1900-01-01 17:25:00      1900-01-01 17:29:00       0
57406   1900-01-01 12:26:00      1900-01-01 12:30:00       0
57407   1900-01-01 06:57:00      1900-01-01 06:59:00       0
57408   1900-01-01 08:53:00      1900-01-01 08:59:00       0
57409   1900-01-01 12:57:00      1900-01-01 12:59:00       0

[57410 rows x 3 columns]


In [ ]:

# Reconvertir les colonnes "Scheduled departure time" et "Actual departure time" pour ne garder que l'heure 
df['Scheduled departure time'] = df['Scheduled departure time'].dt.time
df['Actual departure time'] = df['Actual departure time'].dt.time

# Afficher les colonnes "Actual departure time", "Scheduled departure time" et "Retard"
print(df[['Actual departure time', 'Scheduled departure time', 'Retard']])



      Actual departure time Scheduled departure time  Retard
0                  07:30:00                 07:30:00       0
1                  12:24:00                 12:30:00       0
2                  13:11:00                 12:00:00       1
3                  19:26:00                 19:30:00       0
4                  10:25:00                 10:30:00       0
...                     ...                      ...     ...
57405              17:25:00                 17:29:00       0
57406              12:26:00                 12:30:00       0
57407              06:57:00                 06:59:00       0
57408              08:53:00                 08:59:00       0
57409              12:57:00                 12:59:00       0

[57410 rows x 3 columns]
